In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url)

## parse html ##

In [5]:
soup = BeautifulSoup(html)

In [6]:
ths = soup.thead.tr.find_all('th')
columns = []
for th in ths:
    columns.append(th.string.strip())
columns

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
trs = soup.tbody.find_all('tr')
data = []
for tr in trs:
    item = []
    for td in tr.find_all('td'):
        if td.a:
            item.append(td.a.string.strip())
        else:
            item.append(td.string.strip())
    data.append(item)

In [8]:
df = pd.DataFrame(data)

In [9]:
df.columns = columns

## remove Not assigned rows ##

In [10]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
df.shape

(211, 3)

## Download the Geospatial_data ##

In [33]:
Geospatial_data = pd.read_csv("http://cocl.us/Geospatial_data")

## merge two dataframes ##

In [37]:
Geospatial_data.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
Geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
df_merge_col = pd.merge(df, Geospatial_data, on='Postcode')
df_merge_col.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


## Cluster Neighborhoods ##

In [41]:
from sklearn.cluster import KMeans

In [51]:
data = df_merge_col.drop(columns=['Neighbourhood', 'Borough', 'Postcode'])
data.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.654260,-79.360636
4,43.718518,-79.464763


In [52]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(data)

In [54]:
kmeans.labels_[0:10] 

array([2, 2, 4, 4, 0, 0, 4, 1, 2, 2], dtype=int32)